In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))
import os

In [2]:
import hail as hl

localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'
hl.init(tmp_dir=localfs_path, local_tmpdir=localfs_path, 
        spark_conf={'spark.driver.memory': '15G', 'spark.executor.memory': '30G'})

2022-09-28 12:36:27 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-09-28 12:36:29 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.3, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-09-28 12:36:29 WARN  SparkConf:69 - Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


Running on Apache Spark version 3.1.2
SparkUI available at http://p1985.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/analysis/hail-20220928-1236-0.2.97-937922d7f46c.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats
import seaborn as sns

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [6]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full-healthy.mt')

In [7]:
sport_pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])

2022-08-18 22:01:26 Hail: INFO: Reading table to impute column types(0 + 1) / 1]
2022-08-18 22:01:27 Hail: INFO: Finished type imputation
  Loading field 'sport' as type str (imputed)
  Loading field 'type' as type str (imputed)
  Loading field 'age' as type int32 (imputed)
  Loading field 'sample_id' as type str (imputed)


In [14]:
mt = mt.filter_cols(mt.group == "sport")
mt.write(localfs_path+'sport.mt')

2022-08-18 22:18:07 Hail: INFO: wrote matrix table with 2639210 rows and 100 columns in 14999 partitions to /localfs/25675054/sport.mt


In [15]:
mt = hl.read_matrix_table(localfs_path+'sport.mt')
mt.naive_coalesce(1000).write(localfs_path+'sport-repart.mt')

2022-08-18 22:37:49 Hail: INFO: wrote matrix table with 2639210 rows and 100 columns in 1000 partitions to /localfs/25675054/sport-repart.mt


In [18]:
mt = hl.read_matrix_table(localfs_path+'sport-repart.mt')

In [19]:
mt = hl.variant_qc(mt)
mt.checkpoint(localfs_path+'qc.mt')

2022-08-18 23:06:56 Hail: INFO: wrote matrix table with 2639210 rows and 100 columns in 1000 partitions to /localfs/25675054/qc.mt


In [24]:
mt = mt.filter_rows(hl.is_snp(
    mt.alleles[0], mt.alleles[1])
                   )
mt.checkpoint(localfs_path+'snps.mt')

2022-08-18 23:39:51 Hail: INFO: wrote matrix table with 2397038 rows and 100 columns in 1000 partitions to /localfs/25675054/snps.mt


In [25]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [26]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/start-here-to-delete.mt')

2022-08-18 23:56:14 Hail: INFO: wrote matrix table with 2018543 rows and 100 columns in 1000 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/start-here-to-delete.mt


In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/start-here-to-delete.mt')

In [ ]:
mt = mt.annotate_rows(
    fisher_vs_pl = hl.contingency_table_test(
            mt.variant_qc['AC'][1],
            mt.variant_qc['AC'][0],
            mt.polish_af['AC'][0],
            ((mt.polish_af['AN'])-(mt.polish_af['AC'][0])),
            100
        )
    )
mt.checkpoint(localfs_path+'stat3.mt')

In [ ]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/stat-to-delete.mt')

In [13]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/stat-to-delete.mt')

In [22]:
ht = mt.rows()
ht = ht.filter(
    hl.is_defined(ht.fisher_vs_pl)
)

ht.write(
    '/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/fisher-vs-polish-all-variants-results.ht'
)

2022-08-19 20:13:48 Hail: INFO: wrote table with 1824754 rows in 1000 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/fisher-vs-polish-all-variants-results.ht


In [4]:
ht = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/fisher-vs-polish-all-variants-results.ht')

In [5]:
p = ht.order_by(ht.fisher_vs_pl.p_value).take(100)[99] #4.9963153685774515e-17

In [8]:
p['fisher_vs_pl']['p_value']

4.9963153685774515e-17

In [9]:
to_export = ht.filter(ht.fisher_vs_pl.p_value <= 4.9963153685774515e-17)

In [13]:
to_export.write(localfs_path+'temp.mt')
to_export = hl.read_table(localfs_path+'temp.mt')

2022-09-28 12:45:24 Hail: INFO: wrote table with 100 rows in 1000 partitions to /localfs/27672506/temp.mt


In [14]:
to_export = to_export.transmute(
    DP_stats = to_export.dp_qc,
    GQ_stats = to_export.gq_qc,
    hwe_controls_p_value = to_export.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
    hwe_sport_p_value = to_export.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
    gnomad_v3_nfe_af = to_export.gnomad_v_3_1.freq.AF[2],
    gnomad_v3_nfe_homozygote_count = to_export.gnomad_v_3_1.freq.homozygote_count[2],
    polish_genomes_af = to_export.polish_af.AF,
    cadd = to_export.cadd.cadd_score,
    within_gene = to_export.within_gene,
    hpo = to_export.hpo,
    fisher_sport_vs_polish = to_export.fisher_vs_pl,
    het_sport = to_export.het_non_refs.get('sport', 0),
    het_controls = to_export.het_non_refs.get('control', 0),
    hom_ref_sport = to_export.hom_refs.get('sport', 0),
    hom_ref_controls = to_export.hom_refs.get('control', 0),
    hom_var_sport = to_export.hom_non_refs.get('sport', 0),
    how_var_controls = to_export.hom_non_refs.get('control',0),
    het_endurance = to_export.het_non_refs_e_vs_s.get('endurance', 0),
    het_speed = to_export.het_non_refs_e_vs_s.get('speed', 0),
    hom_ref_endurance = to_export.hom_refs_e_vs_s.get('endurance', 0),
    hom_ref_speed = to_export.hom_refs_e_vs_s.get('speed', 0),
    hom_var_endurance = to_export.hom_non_refs_e_vs_s.get('endurance', 0),
    hom_var_speed = to_export.hom_non_refs_e_vs_s.get('speed',0),
    most_severe_consequence = to_export.vep.vep.most_severe_consequence,
    transcript_consequences = to_export.vep.vep.transcript_consequences,
    intergenic_consequences = to_export.vep.vep.intergenic_consequences,
    motif_feature_consequences = to_export.vep.vep.motif_feature_consequences,
    regulatory_feature_consequences = to_export.vep.vep.regulatory_feature_consequences)

to_export.write(localfs_path+'to-export.mt')

2022-09-28 12:45:30 Hail: INFO: wrote table with 100 rows in 1000 partitions to /localfs/27672506/to-export.mt


In [15]:
to_export = hl.read_table(localfs_path+'to-export.mt')
to_export = to_export.to_pandas()

to_export.to_csv('top_100_sport_vs_polish.csv')

In [55]:
#only keep intragenic variants with cadd > 30
ht = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/fisher-vs-polish-all-variants-results.ht')
ht = ht.filter(ht.within_gene == hl.empty_array(hl.tstr), keep = False)
ht = ht.filter(ht.cadd.cadd_score > 20)
ht.write(localfs_path+'intragenic-20.mt')

2022-08-19 21:46:48 Hail: INFO: wrote table with 5959 rows in 1000 partitions to /localfs/25677471/intragenic-20.mt


In [56]:
ht = hl.read_table(localfs_path+'intragenic-20.mt') #5959

In [66]:
ht = ht.filter((ht.variant_qc.AC[1]+ht.polish_af.AC[0])>5) 
p = ht.order_by(ht.fisher_vs_pl.p_value).take(100)[99] #0.01597463798702992

In [70]:
to_export = ht.filter(ht.fisher_vs_pl.p_value <= 0.01597463798702992)

In [ ]:
to_export = to_export.transmute(
    DP_stats = to_export.dp_qc,
    GQ_stats = to_export.gq_qc,
    hwe_controls_p_value = to_export.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
    hwe_sport_p_value = to_export.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
    gnomad_v3_nfe_af = to_export.gnomad_v_3_1.freq.AF[2],
    gnomad_v3_nfe_homozygote_count = to_export.gnomad_v_3_1.freq.homozygote_count[2],
    polish_genomes_af = to_export.polish_af.AF,
    cadd = to_export.cadd.cadd_score,
    within_gene = to_export.within_gene,
    hpo = to_export.hpo,
    fisher_sport_vs_polish = to_export.fisher_vs_pl,
    het_sport = to_export.het_non_refs.get('sport', 0),
    het_controls = to_export.het_non_refs.get('control', 0),
    hom_ref_sport = to_export.hom_refs.get('sport', 0),
    hom_ref_controls = to_export.hom_refs.get('control', 0),
    hom_var_sport = to_export.hom_non_refs.get('sport', 0),
    how_var_controls = to_export.hom_non_refs.get('control',0),
    het_endurance = to_export.het_non_refs_e_vs_s.get('endurance', 0),
    het_speed = to_export.het_non_refs_e_vs_s.get('speed', 0),
    hom_ref_endurance = to_export.hom_refs_e_vs_s.get('endurance', 0),
    hom_ref_speed = to_export.hom_refs_e_vs_s.get('speed', 0),
    hom_var_endurance = to_export.hom_non_refs_e_vs_s.get('endurance', 0),
    hom_var_speed = to_export.hom_non_refs_e_vs_s.get('speed',0),
    most_severe_consequence = to_export.vep.vep.most_severe_consequence,
    transcript_consequences = to_export.vep.vep.transcript_consequences,
    intergenic_consequences = to_export.vep.vep.intergenic_consequences,
    motif_feature_consequences = to_export.vep.vep.motif_feature_consequences,
    regulatory_feature_consequences = to_export.vep.vep.regulatory_feature_consequences)

to_export.write(localfs_path+'to-export-2.mt')

In [72]:
to_export = hl.read_table(localfs_path+'to-export-2.mt')

In [74]:
to_export.export('top_sport_vs_polish_pathogenic.tsv')

2022-08-19 22:26:16 Hail: INFO: merging 1001 files totalling 816.0K...5) / 1000]
2022-08-19 22:26:16 Hail: INFO: while writing:
    top_sport_vs_polish_pathogenic.tsv
  merge time: 114.841ms


In [ ]:
to_export